In [ ]:
class BattleshipGame {
    constructor() {
        this.players = ['player1', 'player2'];
        this.currentPlayerIndex = 0;
        this.placement = {};
        this.shipLengths = {
            'carrier': 5,
            'battleship': 4,
            'cruiser': 3
        };
        this.initializeGame();
        this.missiles = 3; // Each player gets three missiles
    }

    getCurrentPlayer() {
        return this.players[this.currentPlayerIndex];
    }

    getCurrentPlayerIndex() {
        return this.currentPlayerIndex
    }

    switchPlayer() {
        this.currentPlayerIndex = (this.currentPlayerIndex + 1) % this.players.length;
        if (this.allShipsSunk()) {
            alert(`${this.getCurrentPlayer()} wins! All opponent's ships have been destroyed.`);
            return;
        }
        this.initializeGameBoardHTML(); // Always clear the board when switching players
        this.displayBoard(this.placement[this.getCurrentPlayer() + '_matrix'], this.getCurrentPlayer());

        // Check if both players have placed their ships
        if (this.allShipsPlaced()) {
            this.resetMissilesForPlacement(); // Prepare missiles if all ships are placed
        } else {
            this.resetShipsForPlacement(); // Otherwise, continue with ship placement
        }
    }



    initializeGame() {
        this.players.forEach(player => {
            this.placement[player + '_matrix'] = this.createGameBoard();
        });
        this.initializeGameBoardHTML();
        this.displayBoard(this.placement[this.getCurrentPlayer() + '_matrix'], this.getCurrentPlayer());
        this.resetShipsForPlacement(); // Initial placement of ships
    }

    createGameBoard() {
        let board = [];
        for (let i = 0; i < 10; i++) {
            board[i] = new Array(10).fill('.');
        }
        return board;
    }

    areCoordsValid(coords, board) {
        return coords.every(([r, c]) => r >= 0 && r < 10 && c >= 0 && c < 10 && board[r][c] === '.');
    }

    displayBoard(board, player) {
        console.log(`${player}'s board:`);
        console.log('  ' + '0123456789'.split('').join(' '));
        board.forEach((row, index) => {
            console.log(String.fromCharCode(65 + index) + ' ' + row.join(' '));
        });
    }

    resetShipsForPlacement() {
        const shipContainer = document.getElementById('ships');
        shipContainer.innerHTML = `
            <div id="carrier" draggable="true" class="ship">Carrier (5)</div>
            <div id="battleship" draggable="true" class="ship">Battleship (4)</div>
            <div id="cruiser" draggable="true" class="ship">Cruiser (3)</div>
        `;

        const ships = document.querySelectorAll('.ship');
        ships.forEach(ship => {
            ship.addEventListener('dragstart', handleDragStart);
        });
    }

    resetMissilesForPlacement() {
        this.missiles = 3;  // Reset missile count
        const missileContainer = document.getElementById('missiles');
        missileContainer.innerHTML = ''; // Clear previous missiles
        for (let i = 0; i < this.missiles; i++) {
            const missile = document.createElement('div');
            missile.id = 'missile' + i;
            missile.draggable = true;
            missile.classList.add('missile');
            missile.textContent = 'Missile ' + (i + 1);
            missileContainer.appendChild(missile);
            missile.addEventListener('dragstart', handleDragStart);
        }
        // Ensure the game board is visually cleared or updated for missile phase
        this.initializeGameBoardHTML();
        this.displayBoard(this.placement[this.getCurrentPlayer() + '_matrix'], this.getCurrentPlayer());

        // Alert the players whose turn it is to attack
        alert(`${this.getCurrentPlayer()}'s turn to attack`);
    }



    placeShip(shipId, targetCell) {
        const shipLength = this.shipLengths[shipId];
        let [row, col] = [parseInt(targetCell.dataset.row), parseInt(targetCell.dataset.col)];
        const board = this.placement[this.getCurrentPlayer() + '_matrix'];

        let coords = [];
        for (let i = 0; i < shipLength; i++) {
            coords.push([row, col + i]);
        }

        if (this.areCoordsValid(coords, board)) {
            coords.forEach(([r, c]) => board[r][c] = 'S');
            this.displayBoard(board, this.getCurrentPlayer());
            return true;
        } else {
            console.error('Invalid ship placement for ' + shipId);
            return false;
        }
    }

    launchMissile(targetCell) {
        const [row, col] = [parseInt(targetCell.dataset.row), parseInt(targetCell.dataset.col)];
        const opponentMatrix = this.placement[this.players[(this.currentPlayerIndex + 1) % 2] + '_matrix'];
        if (opponentMatrix[row][col] === 'S') {
            console.log('Hit at ' + String.fromCharCode(65 + row) + (col + 1));
            opponentMatrix[row][col] = 'X'; // Mark hit
            targetCell.classList.add('hit');
        } else {
            console.log('Miss at ' + String.fromCharCode(65 + row) + (col + 1));
            opponentMatrix[row][col] = 'M'; // Mark miss
            targetCell.classList.add('miss');
        }
    }

    allShipsSunk() {
        const opponentMatrix = this.placement[this.players[(this.currentPlayerIndex + 1) % 2] + '_matrix'];
        return !opponentMatrix.some(row => row.includes('S'));
    }
    allShipsPlaced() {
        return this.players.every(player => this.placement[player + '_shipsPlaced']);
    }


    initializeGameBoardHTML() {
        const gameBoard = document.getElementById('player1');
        gameBoard.innerHTML = ''; // Clear previous board
        for (let row = 0; row < 10; row++) {
            for (let col = 0; col < 10; col++) {
                const cell = document.createElement('div');
                cell.dataset.row = row;
                cell.dataset.col = col;
                gameBoard.appendChild(cell);
            }
        }
    }
}

document.addEventListener('DOMContentLoaded', function () {
    const game = new BattleshipGame();

    const ships = document.querySelectorAll('.ship');
    ships.forEach(ship => {
        ship.addEventListener('dragstart', handleDragStart);
    });

    const gameBoard = document.getElementById('player1');
    gameBoard.addEventListener('dragover', handleDragOver);
    gameBoard.addEventListener('drop', event => handleDrop(event, game));
});

function handleDragStart(e) {
    e.dataTransfer.setData('text/plain', e.target.id);
}

function handleDragOver(e) {
    e.preventDefault(); // Necessary to allow dropping
}

function handleDrop(e, game) {
    e.preventDefault();
    const itemID = e.dataTransfer.getData('text/plain');
    const targetCell = e.target.closest('div');

    if (itemID.startsWith('missile')) {
        if (targetCell && game.launchMissile(targetCell)) {
            alert('Missile launched!');
            if (game.missiles <= 0) {
                game.switchPlayer(); // Switch to next player if all missiles are used
            }
        } else {
            alert('Missile missed!');
        }
    } else if (targetCell && game.placeShip(itemID, targetCell)) {
        targetCell.appendChild(document.getElementById(itemID));
        document.getElementById(itemID).draggable = false;

        // Check if all ships of the current player are placed
        const ships = document.querySelectorAll('.ship');
        if (Array.from(ships).every(ship => !ship.draggable)) {
            game.placement[game.getCurrentPlayer() + '_shipsPlaced'] = true;
            if (game.getCurrentPlayerIndex() === 0) {
                game.switchPlayer(); // Switch to Player 2 for ship placement
            } else {
                game.resetMissilesForPlacement(); // Start missile phase for Player 1
            }
        }
    } else {
        alert('Invalid placement, try again.');
    }
}



